# Cleaning

In [1]:
import pandas as pd
import numpy as np
import re

# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# pd.reset_option('display.max_rows')
# pd.reset_option('display.max_columns')

In [2]:
ak = pd.read_csv('../datasets/alaska_single_engine.csv', low_memory=False)

In [3]:
ak.dtypes

Unnamed: 0               int64
NtsbNo                  object
EventType               object
Mkey                     int64
EventDate               object
City                    object
State                   object
Country                 object
ReportNo                object
N                       object
HasSafetyRec              bool
ReportType              object
OriginalPublishDate     object
HighestInjuryLevel      object
FatalInjuryCount         int64
SeriousInjuryCount       int64
MinorInjuryCount         int64
ProbableCause           object
EventID                float64
Latitude               float64
Longitude              float64
Make                    object
Model                   object
AirCraftCategory        object
AirportID               object
AirportName             object
AmateurBuilt            object
NumberOfEngines         object
Scheduled               object
PurposeOfFlight         object
FAR                     object
AirCraftDamage          object
WeatherC

In [4]:
ak.isna().sum()

Unnamed: 0                0
NtsbNo                    0
EventType                 0
Mkey                      0
EventDate                 0
City                      0
State                     0
Country                   0
ReportNo               5645
N                         0
HasSafetyRec              0
ReportType               22
OriginalPublishDate     885
HighestInjuryLevel     4044
FatalInjuryCount          0
SeriousInjuryCount        0
MinorInjuryCount          0
ProbableCause          1452
EventID                5659
Latitude                  0
Longitude                 0
Make                      0
Model                     1
AirCraftCategory          0
AirportID              3510
AirportName            3196
AmateurBuilt              0
NumberOfEngines           0
Scheduled              4342
PurposeOfFlight         414
FAR                       0
AirCraftDamage           52
WeatherCondition         20
Operator               2842
ReportStatus              0
RepGenFlag          

In [5]:
ak.shape

(5659, 39)

## Data Cleaning

In [6]:
ak_clean = ak

In [7]:
# drop unnecessary columns
ak_clean = ak_clean.drop(['Unnamed: 0', 'Unnamed: 37', 'DocketUrl', 'DocketPublishDate', 'State', 'OriginalPublishDate', 'NumberOfEngines', 'ReportNo', 'Country', 'EventID', 'ReportStatus', 'RepGenFlag'], axis=1)

In [8]:
def camel_to_snake(column_name):
    result = []
    i = 0

    while i < len(column_name):
        if column_name[i].isupper():
            if i > 0 and not column_name[i - 1].isupper():
                result.append('_')
            result.append(column_name[i].lower())
        else:
            result.append(column_name[i])
        i += 1

    return ''.join(result)

In [9]:
# camel to snake case feature names
ak_clean.columns = [camel_to_snake(column) for column in ak_clean.columns]

In [10]:
ak_clean.rename(columns={'air_craft_category': 'aircraft_category', 'air_craft_damage': 'aircraft_damage'}, inplace=True)

In [11]:
# select multi-aircraft_columns
comma_columns = ['n', 'make', 'model', 'aircraft_category', 'purpose_of_flight', 'far', 'aircraft_damage', 'operator', 'amateur_built']
mask = ak_clean[comma_columns].apply(lambda x: all(',' in val for val in x), axis=1)

In [12]:
mask.value_counts()

False    5590
True       69
Name: count, dtype: int64

In [13]:
# drop multi-aircraft rows
ak_clean = ak_clean[-mask]
ak_clean.reset_index(drop=True, inplace=True)

In [14]:
ak_clean.shape

(5590, 27)

In [15]:
# fill na

ak_clean = ak_clean.fillna({'highest_injury_level': 'None Reported', 'aircraft_damage': 'None Reported'})

In [16]:
ak_clean = ak_clean.fillna({'airport_id': 'No Airport', 'airport_name': 'No Airport'})

In [17]:
ak_clean = ak_clean.fillna({'weather_condition': 'Unknown',
                 'scheduled': 'Unknown',
                 'operator': 'Unknown',
                 'purpose_of_flight': 'Unknown',
                 'report_type': 'Unknown',
                 'probable_cause': 'Unknown',
                 'model': 'Unknown'})

In [18]:
# remap booleans
ak_clean['amateur_built'] = ak_clean['amateur_built'].replace({'FALSE': 0, 'TRUE': 1})

In [19]:
ak_clean['has_safety_rec'] = ak_clean['has_safety_rec'].replace({'False': 0, 'True': 1}).astype(int)

In [20]:
# convert to datetime
ak_clean['event_date'] = pd.to_datetime(ak_clean['event_date'])

In [21]:
ak_clean.head()

,ntsb_no,event_type,mkey,event_date,city,n,has_safety_rec,report_type,highest_injury_level,fatal_injury_count,serious_injury_count,minor_injury_count,probable_cause,latitude,longitude,make,model,aircraft_category,airport_id,airport_name,amateur_built,scheduled,purpose_of_flight,far,aircraft_damage,weather_condition,operator
0,ANC23LA086,ACC,193153,2023-09-24 08:30:00+00:00,Trimble River / Skwentna,N2586R,0,DirectorBrief,None Reported,0,0,0,Unknown,61.775160,-152.15263,CESSNA,182K,AIR,No Airport,Trimble River,0,Unknown,PERS,091,Substantial,Unknown,Unknown
1,ANC23LA084,ACC,193128,2023-09-20 12:00:00+00:00,Bethel,N8192D,0,DirectorBrief,None Reported,0,0,0,Unknown,60.805019,-161.78648,PIPER,PA-18-150,AIR,No Airport,No Airport,0,Unknown,PERS,091,Substantial,Unknown,Unknown
2,ANC23LA080,ACC,193097,2023-09-18 13:00:00+00:00,Homer,N7558H,0,DirectorBrief,None Reported,0,0,0,Unknown,59.646929,-151.49323,CESSNA,A185F,AIR,5BL,HOMER-BELUGA LAKE,0,Unknown,BUS,091,Substantial,VMC,Adventure Airways
3,ANC23LA082,ACC,193105,2023-09-16 16:50:00+00:00,Beaver Creek,N713C,0,DirectorBrief,None Reported,0,0,0,Unknown,64.267579,-147.68704,HELIO,H-295,AIR,No Airport,No Airport,0,NSCH,BUS,135,Substantial,Unknown,WRIGHT AIR SERVICE INC
4,ANC23LA078,ACC,193088,2023-09-16 15:00:00+00:00,NENANA,N907W,0,DirectorBrief,None Reported,0,0,0,Unknown,64.650753,-149.83639,RHODES STEVEN D,SR3500,AIR,No Airport,No Airport,1,Unknown,PERS,091,Substantial,VMC,Unknown


## Feature Engineering

In [22]:
ak_clean['event_year'] = ak_clean['event_date'].dt.year

In [23]:
ak_clean['event_month'] = ak_clean['event_date'].dt.month

In [24]:
ak_clean['event_day'] = ak_clean['event_date'].dt.day

In [25]:
ak_clean['event_time'] = ak_clean['event_date'].dt.time

In [26]:
ak_clean = ak_clean.drop('event_date', axis=1)

In [27]:
# engineer hour
ak_clean['event_hour'] = [x.hour for x in ak_clean['event_time']]

In [28]:
def get_season(month):
    if 3 <= month <= 5:
        return "Spring"
    elif 6 <= month <= 8:
        return "Summer"
    elif 9 <= month <= 11:
        return "Fall"
    else:
        return "Winter"

In [29]:
# engineer season
ak_clean['event_season'] = ak_clean['event_month'].apply(get_season)

In [30]:
# engineer injury bool
ak_clean['has_injury'] = ak_clean.highest_injury_level.apply(lambda x: 0 if x == 'None Reported' else 1)

In [31]:
# engineer aircraft damage bool
ak_clean['has_aircraft_damage'] = ak_clean.aircraft_damage.apply(lambda x: 0 if x == 'None Reported' else 1)

In [32]:
# engineer event type bool
ak_clean['is_accident'] = ak_clean.event_type.apply(lambda x: 0 if x == 'INC' else 1)

In [33]:
# engineer near airport
ak_clean['occurred_near_airport'] = ak_clean.airport_id.apply(lambda x: 0 if x == 'No Airport' else 1)

In [34]:
ak_clean['make'] = ak_clean['make'].str.upper()

In [35]:
ak_clean['model'] = ak_clean['model'].str.upper()

In [36]:
# engineer make_model
ak_clean['make_model'] = ak_clean['make'] + ' ' + ak_clean['model']

In [37]:
# aircraft family patterns
aircraft_patterns = {
    'AERONCA 15AC SEDAN': r'AERONCA [A-Z]?15AC\b',
    'AERONCA 7AC CHAMPION': r'AERONCA 7AC',
    'BEECHCRAFT 1900': r'BEECH 1900',
    'BEECHCRAFT 36 BONANZA': r'BEECH A36|BEECH 36|BEECH T36|BEECH G36',
    'BELL 206': r'BELL 206[A-Z]?\b',
    'CHAMPION/ BELLANCA/ AMERICAN CHAMPION CITABRIA': r'7ECA\b|7GCAA\b|7GCB|7KCAB\b|8KCAB\b|CHAMPION 7EC|CHAMPION 7GC\b',
    'CHAMPION/ BELLANCA/ AMERICAN CHAMPION SCOUT': r'8GCBC\b',
    'CESSNA 140': r'CESSNA [A-Z]?[-\s]*140[A-Z]?\b',
    'CESSNA 150': r'CESSNA [A-Z]?[-\s]*150[A-Z]?\b',
    'CESSNA 152': r'CESSNA [A-Z]?[-\s]*152[A-Z]?\b',
    'CESSNA 170': r'CESSNA [A-Z]?[-\s]*170[A-Z]?\b',
    'CESSNA 175 SKYLARK': r'CESSNA [A-Z]?[-\s]*175[A-Z]?\b',
    'CESSNA 172 SKYHAWK': r'CESSNA [A-Z]?[-\s]*172[A-Z]?\b',
    'CESSNA 177 CARDINAL': r'CESSNA [A-Z]?[-\s]*177[A-Z]*?\b',
    'CESSNA 180 SKYWAGON': r'CESSNA [A-Z]?[-\s]*180[A-Z]?\b',
    'CESSNA 182 SKYLANE': r'CESSNA [A-Z]?[-\s]*182[A-Z]?\b',
    'CESSNA 185 SKYWAGON': r'CESSNA [A-Z]?[-\s]?185[A-Z]?\b',
    'CESSNA 206 STATIONAIR': r'CESSNA [A-Z]?[-\s]*206[A-Z]?\b|CESSNA TU206',
    'CESSNA 207': r'CESSNA [A-Z]?[-\s]*207[A-Z]?\b',
    'CESSNA 208 CARAVAN': r'CESSNA [A-Z]?[-\s]*208[A-Z]?\b',
    'CESSNA 210': r'CESSNA 210',
    'CESSNA 402': r'CESSNA [A-Z]?402[A-Z]?\b',
    'DE HAVILLAND DHC-2 BEAVER': r'DE[\s]*HAVILLAND DHC[-\s]*2\b|DE[\s]*HAVILLAND BEAVER DHC[-\s]*2',
    'DE HAVILLAND DHC-3 OTTER': r'DE[\s]*HAVILLAND DHC[-\s]*3\b|DE[\s]*HAVILLAND OTTER DHC[-\s]*3',
    'DE HAVILLAND DHC-6 TWIN OTTER': r'DE[\s]*HAVILLAND DHC[-\s]*6\b',
    'DOUGLAS DC-3': r'DOUGLAS DC[-\s]*3',
    'DOUGLAS DC-6': r'DOUGLAS DC[-\s]*6',
    'GRUMMAN G-44 WIDGEON': r'GRUMMAN G[-\s]*44',
    'HELIO COURIER': r'HELIO H',
    'LUSCOMBE 8': r'LUSCOMBE 8',
    'PIPER J-3 CUB': r'PIPER J[-\s]*3',
    'PIPER J-5 CUB CRUISER': r'PIPER J[-\s]*5',
    'PIPER PA-11 CUB SPECIAL': r'PIPER\s*PA[-\s]*11[-\s]*\d*[A-Z]?\b',
    'PIPER PA-12 SUPER CRUISER': r'PIPER\s*PA[-\s]*12[-\s]*\d*[A-Z]?\b',
    'PIPER PA-14 FAMILY CRUISER': r'PIPER\s*PA[-\s]*14[-\s]*\d*[A-Z]?\b',
    'PIPER PA-16 CLIPPER': r'PIPER\s*PA[-\s]*16[-\s]*\d*[A-Z]?\b',
    'PIPER PA-18 SUPER CUB': r'PIPER\s*PA[-\s]*18[-\s]*\d*[A-Z]?\b',
    'PIPER PA-20 PACER': r'PIPER\s*PA[-\s]*20[-\s]*\d*[A-Z]?\b',
    'PIPER PA-22 TRI-PACER': r'PIPER\s*PA[-\s]*22[-\s]*\d*[A-Z]?\b',
    'PIPER PA-28 CHEROKEE': r'PIPER\s*PA[-\s]*28[-\s]*\d*[A-Z]?\b',
    'PIPER PA-31 NAVAJO': r'PIPER\s*PA[-\s]*31[-\s]*\d*[A-Z]?\b',
    'PIPER PA-32 CHEROKEE SIX': r'PIPER\s*PA[-\s]*32[-\s]*\d*[A-Z]?\b',
    'STINSON 108': r'STINSON 108',
    'MAULE M-4': r'MAULE M[-\s]*4',
    'MAULE M-5': r'MAULE M[-\s]*5',
    'MAULE M-6': r'MAULE M[-\s]*6',
    'MAULE M-7': r'MAULE M[-\s]*7',
    'TAYLORCRAFT B': r'TAYLORCRAFT B',
    'TAYLORCRAFT F-19 SPORTSMAN': r'TAYLORCRAFT F[-\s]*19',
}

def group_models(make_model):
    grouped_models = []
    for aircraft, pattern in aircraft_patterns.items():
        if pattern and re.search(pattern, make_model):
            grouped_models.append(aircraft)

    if not grouped_models:
        return make_model
    elif len(grouped_models) == 1:
        return grouped_models[0]
    else:
        return make_model

In [38]:
# engineer aircraft families
ak_clean['aircraft_family'] = ak_clean['make_model'].apply(group_models)

In [39]:
ak_clean.model.nunique()

1053

In [51]:
ak_clean.model.value_counts().head(25)

model
PA-18        439
PA-18-150    245
180          226
185          185
PA-12        172
207          135
170B         132
DHC-2        127
172          112
206          106
7GCBC         78
207A          73
PA-22         65
A185F         65
PA-32-300     60
185F          59
150           57
PA-31-350     53
182           50
PA18          49
PA-20         48
8GCBC         44
170           39
U206G         39
U206          36
Name: count, dtype: int64

In [40]:
ak_clean.aircraft_family.nunique()

758

In [42]:
# label uncommon families
aircraft_family_counts = ak_clean['aircraft_family'].value_counts()
uncommon_families = aircraft_family_counts[aircraft_family_counts < 10].index
ak_clean['aircraft_family'] = ak_clean['aircraft_family'].apply(lambda x: 'UNCOMMON FAMILY' if x in uncommon_families else x)

In [43]:
ak_clean.aircraft_family.nunique()

50

In [44]:
ak_clean.aircraft_family.value_counts().head(50)

aircraft_family
UNCOMMON FAMILY                                   961
PIPER PA-18 SUPER CUB                             893
CESSNA 185 SKYWAGON                               409
CESSNA 180 SKYWAGON                               349
CESSNA 206 STATIONAIR                             264
CESSNA 207                                        228
PIPER PA-12 SUPER CRUISER                         196
CESSNA 170                                        193
DE HAVILLAND DHC-2 BEAVER                         191
CESSNA 172 SKYHAWK                                189
PIPER PA-32 CHEROKEE SIX                          158
CHAMPION/ BELLANCA/ AMERICAN CHAMPION CITABRIA    151
PIPER PA-22 TRI-PACER                             136
CESSNA 150                                        108
CESSNA 182 SKYLANE                                100
PIPER PA-31 NAVAJO                                 83
STINSON 108                                        62
PIPER PA-20 PACER                                  57
BELL 206    

In [45]:
ak_clean.isna().sum()

ntsb_no                  0
event_type               0
mkey                     0
city                     0
n                        0
has_safety_rec           0
report_type              0
highest_injury_level     0
fatal_injury_count       0
serious_injury_count     0
minor_injury_count       0
probable_cause           0
latitude                 0
longitude                0
make                     0
model                    0
aircraft_category        0
airport_id               0
airport_name             0
amateur_built            0
scheduled                0
purpose_of_flight        0
far                      0
aircraft_damage          0
weather_condition        0
operator                 0
event_year               0
event_month              0
event_day                0
event_time               0
event_hour               0
event_season             0
has_injury               0
has_aircraft_damage      0
is_accident              0
occurred_near_airport    0
make_model               0
a

In [46]:
ak_clean.dtypes

ntsb_no                   object
event_type                object
mkey                       int64
city                      object
n                         object
has_safety_rec             int64
report_type               object
highest_injury_level      object
fatal_injury_count         int64
serious_injury_count       int64
minor_injury_count         int64
probable_cause            object
latitude                 float64
longitude                float64
make                      object
model                     object
aircraft_category         object
airport_id                object
airport_name              object
amateur_built              int64
scheduled                 object
purpose_of_flight         object
far                       object
aircraft_damage           object
weather_condition         object
operator                  object
event_year                 int32
event_month                int32
event_day                  int32
event_time                object
event_hour

In [47]:
ak_clean.head(200)

,ntsb_no,event_type,mkey,city,n,has_safety_rec,report_type,highest_injury_level,fatal_injury_count,serious_injury_count,minor_injury_count,probable_cause,latitude,longitude,make,model,aircraft_category,airport_id,airport_name,amateur_built,scheduled,purpose_of_flight,far,aircraft_damage,weather_condition,operator,event_year,event_month,event_day,event_time,event_hour,event_season,has_injury,has_aircraft_damage,is_accident,occurred_near_airport,make_model,aircraft_family
0,ANC23LA086,ACC,193153,Trimble River / Skwentna,N2586R,0,DirectorBrief,None Reported,0,0,0,Unknown,61.775160,-152.15263,CESSNA,182K,AIR,No Airport,Trimble River,0,Unknown,PERS,091,Substantial,Unknown,Unknown,2023,9,24,08:30:00,8,Fall,0,1,1,0,CESSNA 182K,CESSNA 182 SKYLANE
1,ANC23LA084,ACC,193128,Bethel,N8192D,0,DirectorBrief,None Reported,0,0,0,Unknown,60.805019,-161.78648,PIPER,PA-18-150,AIR,No Airport,No Airport,0,Unknown,PERS,091,Substantial,Unknown,Unknown,2023,9,20,12:00:00,12,Fall,0,1,1,0,PIPER PA-18-150,PIPER PA-18 SUPER CUB
2,ANC23LA080,ACC,193097,Homer,N7558H,0,DirectorBrief,None Reported,0,0,0,Unknown,59.646929,-151.49323,CESSNA,A185F,AIR,5BL,HOMER-BELUGA LAKE,0,Unknown,BUS,091,Substantial,VMC,Adventure Airways,2023,9,18,13:00:00,13,Fall,0,1,1,1,CESSNA A185F,CESSNA 185 SKYWAGON
3,ANC23LA082,ACC,193105,Beaver Creek,N713C,0,DirectorBrief,None Reported,0,0,0,Unknown,64.267579,-147.68704,HELIO,H-295,AIR,No Airport,No Airport,0,NSCH,BUS,135,Substantial,Unknown,WRIGHT AIR SERVICE INC,2023,9,16,16:50:00,16,Fall,0,1,1,0,HELIO H-295,HELIO COURIER
4,ANC23LA078,ACC,193088,NENANA,N907W,0,DirectorBrief,None Reported,0,0,0,Unknown,64.650753,-149.83639,RHODES STEVEN D,SR3500,AIR,No Airport,No Airport,1,Unknown,PERS,091,Substantial,VMC,Unknown,2023,9,16,15:00:00,15,Fall,0,1,1,0,RHODES STEVEN D SR3500,UNCOMMON FAMILY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,ANC21LA057,ACC,103467,Coffman Cove,N9279Z,0,DirectorBrief,None Reported,0,0,0,The pilot’s impact with a buoy on takeoff whic...,56.008959,-132.84709,DEHAVILLAND,DHC-2-L-20,AIR,KCC,COFFMAN COVE,0,Unknown,POSI,091,Substantial,VMC,Southeast Aviation,2021,7,9,05:40:00,5,Summer,0,1,1,1,DEHAVILLAND DHC-2-L-20,DE HAVILLAND DHC-2 BEAVER
196,ANC21LA058,ACC,103469,Anchorage,N349X,0,DirectorBrief,None Reported,0,0,0,The pilot’s impact with a log while dragging a...,61.161813,-149.98618,MAULE,M-5-210C,AIR,LHD,LAKE HOOD,0,Unknown,PERS,091,Substantial,VMC,Unknown,2021,7,7,14:00:00,14,Summer,0,1,1,1,MAULE M-5-210C,MAULE M-5
197,ANC21LA053,ACC,103436,Kenney Lake,N185YT,0,DirectorBrief,None Reported,0,0,0,The pilot’s failure to maintain adequate clear...,61.731800,-144.94360,CESSNA,185,AIR,PVT,Private,0,Unknown,PERS,091,Substantial,VMC,Unknown,2021,7,5,21:45:00,21,Summer,0,1,1,1,CESSNA 185,CESSNA 185 SKYWAGON
198,ANC21LA052,ACC,103418,Palmer,N5213X,0,DirectorBrief,Serious,0,1,1,The pilot’s selection of unsuitable terrain fo...,61.501454,-149.09937,CHAMPION,7GCBC,AIR,No Airport,No Airport,0,Unknown,PERS,091,Substantial,VMC,Unknown,2021,7,4,11:30:00,11,Summer,1,1,1,0,CHAMPION 7GCBC,CHAMPION/ BELLANCA/ AMERICAN CHAMPION CITABRIA


## Export

In [48]:
ak_clean.to_csv('../datasets/alaska_single_engine_clean.csv', index=False)